In [46]:
from IPython.display import Image #type: ignore

import warnings
warnings.filterwarnings('ignore')

In [19]:
display(Image(url="Data/image.png", width=600, height=400))

In [47]:
import pandas as pd

df = pd.read_csv("Data/reviews-pt-br.csv", encoding='utf-8')
df.head(10)

,codigo,texto,sentimento
0,1,Esse bocejo de pia de cozinha de orçamento mui...,neg
1,2,O Bravo parece indicar que o personagem princi...,neg
2,3,"Durante a Guerra pela Independência do Sul, GE...",pos
3,4,É fora de questão que a verdadeira Anna Anders...,pos
4,5,Concordo totalmente com outro dos revisores aq...,neg
5,6,Obra-prima absoluta de um filme! Boa noite Mr....,pos
6,7,Embora a palavra megalmania seja muito usada p...,pos
7,8,Esta tem que ser a peça mais incrível de porca...,neg
8,9,Eu suponho que todas as piadas internas são o ...,neg
9,10,"Se há um tema deste filme, é que as pessoas po...",pos


In [48]:
df.shape

(44514, 3)

In [49]:
df['sentimento'].unique()

array(['neg', 'pos'], dtype=object)

In [50]:
import nltk
import re
import string
import unicodedata
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
    
def normalize_accents(text):
    return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = normalize_accents(text)
    text = re.sub(re.compile(r" +"), " ",text)
    return " ".join([w for w in text.split()])

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text


def tokenizer(text):
    stop_words = nltk.corpus.stopwords.words("portuguese")
    if isinstance(text, str):
        text = normalize_str(text)
        text = "".join([w for w in text if not w.isdigit()])
        text = word_tokenize(text)
        text = [x for x in text if x not in stop_words]
        text = [y for y in text if len(y) > 2]
        return [t for t in text]
    else:
        return None

In [51]:
df['texto_tratado'] = df['texto'].apply(tokenizer)
df['sentimento_tratado'] = df['sentimento'].map({'pos': 1, 'neg': 0})
df.head(10)

,codigo,texto,sentimento,texto_tratado,sentimento_tratado
0,1,Esse bocejo de pia de cozinha de orçamento mui...,neg,"[bocejo, pia, cozinha, orcamento, baixo, tipo,...",0
1,2,O Bravo parece indicar que o personagem princi...,neg,"[bravo, parece, indicar, personagem, principal...",0
2,3,"Durante a Guerra pela Independência do Sul, GE...",pos,"[durante, guerra, independencia, sul, general,...",1
3,4,É fora de questão que a verdadeira Anna Anders...,pos,"[questao, verdadeira, anna, anderson, nao, pri...",1
4,5,Concordo totalmente com outro dos revisores aq...,neg,"[concordo, totalmente, outro, revisores, aqui,...",0
5,6,Obra-prima absoluta de um filme! Boa noite Mr....,pos,"[obra, prima, absoluta, filme, boa, noite, tom...",1
6,7,Embora a palavra megalmania seja muito usada p...,pos,"[embora, palavra, megalmania, usada, descrever...",1
7,8,Esta tem que ser a peça mais incrível de porca...,neg,"[peca, incrivel, porcaria, cinematografica, as...",0
8,9,Eu suponho que todas as piadas internas são o ...,neg,"[suponho, todas, piadas, internas, sao, fez, m...",0
9,10,"Se há um tema deste filme, é que as pessoas po...",pos,"[tema, deste, filme, pessoas, podem, lidar, di...",1


In [52]:
df.isnull().sum().sum()

0

In [53]:
df['sentimento'].value_counts()

sentimento
neg    22307
pos    22207
Name: count, dtype: int64

In [54]:
df['sentimento_tratado'].value_counts()

sentimento_tratado
0    22307
1    22207
Name: count, dtype: int64

In [55]:
import multiprocessing
import numpy as np

dim_vec = 300
min_count = 10
window = 4
num_workers = multiprocessing.cpu_count()
seed = np.random.seed(42)

In [56]:
from gensim.models import Word2Vec

modelo = Word2Vec(df["texto_tratado"],
                    min_count = min_count, 
                    vector_size = dim_vec, 
                    window = window,
                    seed = seed, #type: ignore
                    workers = num_workers,
                    sg = 1)

In [57]:
print("Tamanho do vocabulário do Word2Vec: ", len(modelo.wv))

print(modelo.wv.most_similar(positive = ['show', 'movie'], negative = ['home'], topn = 3)) # similaridade considerando exemplos positivos e negativos


Tamanho do vocabulário do Word2Vec:  30443
[('shows', 0.5126184225082397), ('programa', 0.5022436380386353), ('scary', 0.41896867752075195)]


In [58]:
def meanVector(model,phrase):
    vocab = list(model.wv.index_to_key)
    phrase = " ".join(phrase)
    phrase = [x for x in word_tokenize(phrase) if x in vocab]
    
    if phrase == []:
        vetor = [0.0]*dim_vec 
    else:
        vetor = np.mean([model.wv[word] for word in phrase],axis=0)
    return vetor

In [59]:
def createFeatures(base): 
    features = [meanVector(modelo,base['texto_tratado'][i])for i in range(len(base))]
    return features

In [60]:
labels = np.array(df['sentimento'])
labels_tratado = np.array(df['sentimento_tratado'])

In [61]:
df = createFeatures(df)

In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size=0.2, random_state=42)
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(df, labels_tratado, test_size=0.2, random_state=42)

In [40]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10, 100], 'gamma': ['scale', 'auto', 0.1, 0.01, 0.001]}
grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
print("Best Parameters:", best_params)

y_pred = best_model.predict(X_test)

Best Parameters: {'C': 10, 'gamma': 'scale'}


In [41]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Weighted F1-score:", metrics.f1_score(y_test, y_pred, average='weighted'))

Accuracy: 0.8845333033808829
Weighted F1-score: 0.8845335364604024


In [44]:
from sklearn.neural_network import MLPClassifier

param_grid = {
    'hidden_layer_sizes': [(50, 50), (100,)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
}


grid_search = GridSearchCV(MLPClassifier(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params_mlp = grid_search.best_params_
best_model_mlp = grid_search.best_estimator_

print("Best Parameters:", best_params_mlp)

y_pred_mlp = best_model_mlp.predict(X_test)

Best Parameters: {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (100,)}


In [45]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_mlp))
print("Weighted F1-score:", metrics.f1_score(y_test, y_pred_mlp, average='weighted'))

Accuracy: 0.8830731214197461
Weighted F1-score: 0.882994506712377
